In [80]:
import pandas as pd
import requests
import json

In [50]:
dia_ontem = pd.Timestamp.today() -pd.Timedelta("1 days")
dia_ontem = dia_ontem.strftime('%Y-%m-%d')
hoje = pd.Timestamp.today().strftime('%Y-%m-%d')
dia_ontem

'2024-06-28'

In [107]:
base_url = "https://dadosabertos.camara.leg.br/api/v2"

endpoint = "/proposicoes"

params = {
    "keywords": "Energia,Solar",
    "dataInicio": dia_ontem,
    "ano": [2024],
    "dataFim": hoje,
    "ordenarPor": "id"
}

headers = {
    "Accept": "application/json"
}

response = requests.get(base_url + endpoint, params=params, headers=headers)

projetos_lista = []
if response.status_code == 200:
    dados = response.json()
    proposicoes = dados['dados']
    for proposicao in proposicoes:
        url_procura_id = f"https://dadosabertos.camara.leg.br/api/v2/proposicoes/{proposicao['id']}"
        projeto_id = requests.get(url_procura_id)
        resposta_projeto = projeto_id.json()
        projeto_dict = {
            "ID": resposta_projeto['dados']['id'],
            "Tipo": resposta_projeto['dados']['siglaTipo'],
            "Ano": resposta_projeto['dados']['ano'],
            "Ementa": resposta_projeto['dados']['ementa'],
            "Data Inicio": resposta_projeto['dados']['dataApresentacao'],
            "Ultma Alteracao": resposta_projeto['dados']['statusProposicao']['dataHora'],
            "Palavras Chaves": resposta_projeto['dados']['keywords'],
            "Link": f"https://www.camara.leg.br/proposicoesWeb/fichadetramitacao?idProposicao={resposta_projeto['dados']['id']}"
        }
        projetos_lista.append(projeto_dict)
        
else:
    print(f"Erro na requisição: {response.status_code}")
relatorio_df = pd.DataFrame(projetos_lista)
relatorio_df.to_excel("Projetos-Novos.xlsx", index=False)